In [ ]:
import matplotlib as mpl
import numpy as np
mpl.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [ ]:
import pandas as pd
import re 
summary_data = pd.read_pickle('data/summary.pickle')

Find all columns with score

In [ ]:
metrics_columns = [f for f in summary_data.columns if not re.match(r"(?:fold|model|data|preproc)", f)]
features_columns = [f for f in summary_data.columns if re.match(r"(?:fold|model|data|preproc)", f)]
print(metrics_columns, features_columns)

Find mean, max, min, std for metrics

In [ ]:
%time
for k in metrics_columns:
    summary_data[str(k) + "_mean"] = summary_data[str(k)].apply(pd.Series).mean(axis=1)
    summary_data[str(k) + "_min"] = summary_data[str(k)].apply(pd.Series).min(axis=1)
    summary_data[str(k) + "_max"] = summary_data[str(k)].apply(pd.Series).max(axis=1)
    summary_data[str(k) + "_std"] = summary_data[str(k)].apply(pd.Series).std(axis=1)


In [ ]:
summary_data.head(13)

In [ ]:
summary_data['mean_absolute_error']

combine model's detail in one column

In [ ]:
def unit_data(x):
    x_all = ''
    for x_i in x:
        if x_i != 'nan':
            x_all += x_i
            x_all += '\n'
    return x_all        

In [ ]:
summary_data['label'] = summary_data[features_columns].astype(str).apply(unit_data,1)

sort dataframe by mean value

In [ ]:
sort_summary_data = summary_data.sort_values(by=['mean_squared_error_mean'], ascending=True)

# MAE metric 

In [ ]:
metric_mae = sort_summary_data['mean_absolute_error'].apply(pd.Series).fillna(0).values
r_metric_mae = [list(x) for x in zip(*metric_mae)]

In [ ]:
sns.set(rc={'figure.figsize':(16,10)})
# Show each distribution with violins
ax = sns.violinplot(data=np.array(r_metric_mae))
ax.set_xticklabels(sort_summary_data['label'].values)
plt.xticks(rotation=90)

# RMSE metric 

In [ ]:
metric_rmse = sort_summary_data['mean_squared_error'].apply(pd.Series).fillna(0).values
r_metric_rmse = [list(x) for x in zip(*metric_rmse)]

In [ ]:
sns.set(rc={'figure.figsize':(16,10)})
# Show each distribution with violins
ax = sns.violinplot(data=np.array(r_metric_rmse))
ax.set_xticklabels(sort_summary_data['label'].values)
plt.xticks(rotation=90)